WORKS BEST WITH LOCAL PYTHON OR JUPYTER SERVER RUNNING IN . (dot)

    jupyter notebook --no-browser


In [29]:
from SimObject import SimObject
from CoinSimulator import CoinSimulator
from CoinSimulator import CoinFlipSimulator
from CoinSimulator import Coin
import sys
import logging

### SET LOG LEVEL
LOGLEVEL=logging.ERROR
### FORCE LOGGING FOR LOCAL TESTING
FORCE=True

logging.basicConfig(filename=f"{__name__}.log", level=LOGLEVEL, format='%(asctime)s - %(levelname)s - %(message)s', force=FORCE)
def sim_test(test_id="TEST"):
    logging.info(f"\n### coins ### {test_id} ######################")
def is_positive_int(value):
    try:
        num = int(value)
        return num > 0
    except ValueError:
        return False



print (__name__)
if __name__ == "__main__":
    sim_test("ARGUMENTS")
    logging.info(f"Number of arguments: {len(sys.argv)}")
    logging.info(f"Argument List: {sys.argv}")

    try:
        logging.debug("sys.argv[1]:"+sys.argv[1])
        logging.debug("sys.argv[2]:"+sys.argv[2])
        NUMBER_OF_OBJECTS = int(sys.argv[1])
        NUMBER_OF_SIMULATIONS = int(sys.argv[2])
    except:
        NUMBER_OF_OBJECTS = 3
        NUMBER_OF_SIMULATIONS = 10
    logging.info(f"NUMBER_OF_OBJECTS: {NUMBER_OF_OBJECTS}")
    logging.info(f"NUMBER_OF_SIMULATIONS: {NUMBER_OF_SIMULATIONS}")




__main__


In [32]:
def coin_simulator(number_of_bits=1, number_of_tests=10):
    sim_test("COIN SIMULATOR")
    print(f"coin_simulator({number_of_bits},{number_of_tests}):")
    results = CoinSimulator(number_of_bits).simulate(number_of_tests)
    logging.debug(f"Results: {results}")
    return results

for state, result in coin_simulator(NUMBER_OF_OBJECTS,NUMBER_OF_SIMULATIONS).items():
    print(f"state: {state} result: {result}")

coin_simulator(3,10):
state: coin-0-HEADS result: 5
state: coin-0-TAILS result: 5
state: coin-1-HEADS result: 5
state: coin-1-TAILS result: 5
state: coin-2-HEADS result: 4
state: coin-2-TAILS result: 6


In [31]:
def coin_flip_simulator(number_of_coins=1, number_of_tests=10):
    sim_test("COIN FLIP SIMULATOR")
    print(f"coin_flip_simulator({number_of_coins},{number_of_tests}):")
    sim = CoinFlipSimulator()
    results = {}
    for i in range(0, number_of_coins):
        c = Coin("HEADS",f"coin-{i}")
        for state, state_object in c.get_states().items():
            sim.add_event(state_object[SimObject.STATE], state_object[SimObject.PROBABILITY])
        results[c.NAME] = sim.simulate(number_of_tests)
    logging.debug(f"Results: {results}")
    return results

for coin, result in coin_flip_simulator(number_of_coins=3,number_of_tests=1000).items():
    print(f"coin: {coin}")
    for k,v in result.items():
        print(f"\t{k}:{v}")



coin_flip_simulator(3,1000):
coin: coin-0
	heads:516
	tails:484
coin: coin-1
	heads:493
	tails:507
coin: coin-2
	heads:505
	tails:495


In [82]:
from ProbabilisticSimulator import ProbabilisticSimulator

def simulate_object_probability(
        probability_list=[
            {"probability":.50},
            {"probability":.50}]
        ,simulations=10**2
        ,test_cycles=1):
    print(f"simulate_object_probability({test_cycles},{simulations}):")
    sim = ProbabilisticSimulator()
    for i in range(0, test_cycles):
        upper_limit = 1.0
        cumulative_probability = 0.0
        for ndx, state_object in enumerate(probability_list):
            cumulative_probability += state_object["probability"]
            # print(f"+{state_object["probability"]}={cumulative_probability}")
            if cumulative_probability > upper_limit:
                raise ValueError(f"list of probabilities does not total {upper_limit}: +{state_object["probability"]} gives {cumulative_probability}")
            sim.add_event(f"state-{ndx}-{100*state_object["probability"]}%", state_object["probability"])
        if cumulative_probability < upper_limit:
            raise ValueError(f"Upper limit not reached {upper_limit}: @ {cumulative_probability}")
        results = {f"object-{i}":sim.simulate(simulations)}

        for item,result in results.items():
            print(f"{item} ran {simulations:,} of trials over {len(probability_list):,} states")
            for k,v in result.items():
                v = f"{round(100*(v/simulations), len(str(simulations))-1)}%"
                print(f"\t{k}\t{v}")
    print()
# END FUNCTION

simulate_object_probability()
simulate_object_probability(simulations=10**5)
l = [
            {"probability":.10},
            {"probability":.20},
            {"probability":.15},
            {"probability":.12},
            {"probability":.21},
            {"probability":.12},
            {"probability":.08},
            {"probability":.02},
            ]
simulate_object_probability(l,simulations=10**6,test_cycles=1)


simulate_object_probability(1,100):
object-0 ran 100 of trials over 2 states
	state-0-50.0%	43.0%
	state-1-50.0%	57.0%

simulate_object_probability(1,100000):
object-0 ran 100,000 of trials over 2 states
	state-0-50.0%	50.047%
	state-1-50.0%	49.953%

simulate_object_probability(1,1000000):
object-0 ran 1,000,000 of trials over 8 states
	state-0-10.0%	9.9978%
	state-1-20.0%	19.9827%
	state-2-15.0%	15.0344%
	state-3-12.0%	11.9367%
	state-4-21.0%	21.0296%
	state-5-12.0%	11.9994%
	state-6-8.0%	8.0002%
	state-7-2.0%	2.0192%

